In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
!pip install holidays
import holidays
%matplotlib inline

In [5]:
#from google.colab import drive
#drive.mount('/content/gdrive')
sales = pd.read_csv('/content/gdrive/My Drive/College/DATA 401/Iowa-Liquor-Sales/GrossSalesDay.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#from google.colab import drive
#drive.mount('/My Drive/College/DATA 401')
#sales = pd.read_csv('/My Drive/College/DATA 401/Iowa-Liquor-Sales/GrossSalesDays.csv')

spellingMapping = {
    "clearlake": "clear lake",
    "dewitt": "de witt",
    "guttenburg" : "guttenberg",
    "grand mounds": "grand mound",
    "jewell": "jewell junction",
    "kellog": "kellogg",
    "ottuwma": "ottumwa",
    "otumwa": "ottumwa",
    "lemars": "le mars",
    "leclaire": "le claire"
}   

sales['City'] = sales['City'].map(lambda x: x.lower().replace("'", "").strip())

sales['City'] = sales['City'].map(lambda x: spellingMapping.get(x, x))
sales['City'] = sales['City'].map(lambda x: x.replace('mt', 'mount'))
sales['City'] = sales['City'].map(lambda x: x.replace('saint', 'st.'))
sales['City'] = sales['City'].map(lambda x: re.sub('^ft.', 'fort', x))
sales['City'] = sales['City'].map(lambda x: re.sub('^st ', 'st. ', x))

In [0]:
census = pd.read_csv("/content/gdrive/My Drive/College/DATA 401/Iowa-Liquor-Sales/CensusData.csv")
census['City'] = census['City'].map(lambda x: x.lower().replace("'", "").strip())

In [30]:
censusunique = np.unique(census['City'])
for city in np.unique(sales['City']):
    if city not in censusunique:
        print(city)

colorado springs


Colorado Springs isn't a city in Iowa. We only lose 1 row by keeping Colorado Springs out, so we'll leave it out. 

In [31]:
len(sales.index)

121045

In [0]:
merged = sales.merge(census, on='City', how='inner')
merged = merged.drop("Unnamed: 0", axis = 1)

In [0]:
merged['Volume Per Capita'] = merged['Volume Sold (Liters)'] / merged['Population']
merged['Bottles Per Capita'] = merged['Bottles Sold'] / merged['Population']
merged['Sales Per Capita'] = merged['Sale (Dollars)'] / merged['Population']

In [34]:
len(merged.columns)

23

In [35]:
merged.head()

,City,Date,aggVODKA,aggRUM,aggBRANDIES,aggSCHNAPPS,aggWHISKIES,aggTEQUILA,aggSCOTCH,aggOTHER,...,Bottles Sold,Sale (Dollars),Population,HousingUnits,AreaSQMT,DensitySQMP,DensitySQMH,Volume Per Capita,Bottles Per Capita,Sales Per Capita
0,ackley,01/04/2016,0.133333,0.066667,0.000000,0.000000,0.533333,0.000000,0.0,0.133333,...,118,1858.58,1589,789,2.48,648.6,322.0,0.071743,0.074261,1.169654
1,ackley,01/05/2015,0.312500,0.250000,0.000000,0.062500,0.250000,0.000000,0.0,0.125000,...,158,1953.04,1589,789,2.48,648.6,322.0,0.103203,0.099434,1.229100
2,ackley,01/06/2014,0.190476,0.047619,0.047619,0.047619,0.428571,0.047619,0.0,0.142857,...,126,1260.57,1589,789,2.48,648.6,322.0,0.092196,0.079295,0.793310
3,ackley,01/07/2013,0.125000,0.000000,0.062500,0.000000,0.500000,0.000000,0.0,0.187500,...,96,955.93,1589,789,2.48,648.6,322.0,0.062146,0.060415,0.601592
4,ackley,01/08/2013,0.250000,0.250000,0.000000,0.000000,0.250000,0.125000,0.0,0.000000,...,72,686.59,1589,789,2.48,648.6,322.0,0.038704,0.045312,0.432089


In [36]:
len(merged.City.unique())

418

In [0]:
one_hot_city = pd.get_dummies(merged.City)
merged = pd.merge(merged, one_hot_city, left_index=True, right_index=True)


merged.Date = pd.to_datetime(merged.Date, infer_datetime_format=True)
# The day of the week with Monday=0, Sunday=6
merged['Dayofweek'] = merged.Date.dt.dayofweek
# The week ordinal of the year
merged['Week'] = merged.Date.dt.weekofyear
merged['Month'] = merged.Date.dt.month

#print("merged.Date = {} and type = {}".format(merged.Date, type(merged.Date)))
us_holidays = holidays.UnitedStates()
merged['Holiday'] = merged.apply(lambda x: x['Date'] in us_holidays, axis =1)

In [38]:
merged.head()

,City,Date,aggVODKA,aggRUM,aggBRANDIES,aggSCHNAPPS,aggWHISKIES,aggTEQUILA,aggSCOTCH,aggOTHER,...,winfield,winterset,winthrop,woodbine,woodward,zwingle,Dayofweek,Week,Month,Holiday
0,ackley,2016-01-04,0.133333,0.066667,0.000000,0.000000,0.533333,0.000000,0.0,0.133333,...,0,0,0,0,0,0,0,1,1,False
1,ackley,2015-01-05,0.312500,0.250000,0.000000,0.062500,0.250000,0.000000,0.0,0.125000,...,0,0,0,0,0,0,0,2,1,False
2,ackley,2014-01-06,0.190476,0.047619,0.047619,0.047619,0.428571,0.047619,0.0,0.142857,...,0,0,0,0,0,0,0,2,1,False
3,ackley,2013-01-07,0.125000,0.000000,0.062500,0.000000,0.500000,0.000000,0.0,0.187500,...,0,0,0,0,0,0,0,2,1,False
4,ackley,2013-01-08,0.250000,0.250000,0.000000,0.000000,0.250000,0.125000,0.0,0.000000,...,0,0,0,0,0,0,1,2,1,False


In [39]:
list(merged)

['City',
 'Date',
 'aggVODKA',
 'aggRUM',
 'aggBRANDIES',
 'aggSCHNAPPS',
 'aggWHISKIES',
 'aggTEQUILA',
 'aggSCOTCH',
 'aggOTHER',
 'aggLIQUEURS',
 'numRecords',
 'Volume Sold (Liters)',
 'Bottles Sold',
 'Sale (Dollars)',
 'Population',
 'HousingUnits',
 'AreaSQMT',
 'DensitySQMP',
 'DensitySQMH',
 'Volume Per Capita',
 'Bottles Per Capita',
 'Sales Per Capita',
 'ackley',
 'adair',
 'adel',
 'afton',
 'akron',
 'albert city',
 'albia',
 'albion',
 'alburnett',
 'alden',
 'algona',
 'allison',
 'alta',
 'alta vista',
 'alton',
 'altoona',
 'alvord',
 'amana',
 'ames',
 'anamosa',
 'anita',
 'ankeny',
 'anthon',
 'arlington',
 'armstrong',
 'arnolds park',
 'atkins',
 'atlantic',
 'audubon',
 'aurelia',
 'avoca',
 'baldwin',
 'bancroft',
 'baxter',
 'bedford',
 'belle plaine',
 'bellevue',
 'belmond',
 'bettendorf',
 'bevington',
 'blairstown',
 'bloomfield',
 'blue grass',
 'bondurant',
 'boone',
 'britt',
 'brooklyn',
 'buffalo',
 'buffalo center',
 'burlington',
 'bussey',
 'camanc

In [0]:
merged.to_csv("/content/gdrive/My Drive/College/DATA 401/Iowa-Liquor-Sales/GrossSalesAggregated_Final.csv", index=False)